In [1]:
import os

import xarray as xr
import numpy as np


import vtk as vtk
from tvtk.api import tvtk, write_data

from pathlib import Path

In [2]:
data_dir = Path(os.path.dirname(os.path.abspath("__file__"))).parent.joinpath('data')

# Dataset for ground water model (concentrations)
# Dataset ds is ordered with (z,y,x) coordinates
ds = xr.open_dataset(data_dir.joinpath('concentratie_data_gw_model.nc'))
# Replace negative concentrations (due to model errors) with 0
ds = ds.where((ds.conc >= 0) | ds.isnull(), other = 0)

In [3]:
# Create vtk pipeline
data = np.array(ds.conc)
data = data.swapaxes(0,2)
data[np.isnan(data)] = -10 # Replace NaN values for it did not load properly
r = tvtk.RectilinearGrid()
r.point_data.scalars = data.ravel(order='F') # Order F due (x,y,z) order of dataset
r.point_data.scalars.name = 'scalars'
r.dimensions = data.shape
r.x_coordinates = np.array(ds.x)
r.y_coordinates = np.array(ds.y)
r.z_coordinates = np.array(ds.z)

In [4]:
write_data(r, str(data_dir.joinpath('conc_no_nan_vtk.vtk')))